In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import os

import statsmodels.api as sms 
from sklearn.model_selection import train_test_split

# **Importar Datos** 

In [7]:
os.getcwd()

'/Users/osgaspar/Documents/PERSONAL/Cedesistemas/2023/Diplomado2023_02/ModelosSupervisados/Clasificacion/ProyectoMora'

In [ ]:
df=pd.read_csv("")